In [ ]:
# Exports QuantumCollocation, NamedTrajectories, and TrajectoryIndexingUtils
using Piccolo
using LinearAlgebra

# Plots
using CairoMakie

In [ ]:
const Units = 1e9
const MHz = 1e6 / Units
const GHz = 1e9 / Units
const ns = 1e-9 * Units
const μs = 1e-6 * Units

const n_qubits = 1
const n_levels = 2

t_f = 10 * ns
n_steps = 101
times = range(0, t_f, n_steps)  # Alternative: collect(0:Δt:t_f)
Δt = times[2] - times[1]

In [ ]:
# Operators
Paulis = Dict(
    "I" => Matrix{ComplexF64}(I, 2, 2),
    "X" => Matrix{ComplexF64}([0 1; 1 0]),
    "Y" => Matrix{ComplexF64}([0 -im; im 0]),
    "Z" => Matrix{ComplexF64}([1 0; 0 -1]),
)
X0=kron(Paulis["X"],Paulis["I"])
Y0=kron(Paulis["Y"],Paulis["I"])
X1=kron(Paulis["I"],Paulis["X"])
Y1=kron(Paulis["I"],Paulis["Y"])

excitation(theta) = exp(-im/2 * theta * (X0 * Y1-Y0 * X1))

In [ ]:
a = [0 1; 0 0]
ad = transpose(a)
a0  = kron(a,Paulis["I"])
ad0  = kron(ad,Paulis["I"])
a1  = kron(Paulis["I"],a)
ad1  = kron(Paulis["I"],ad);

In [ ]:
H_drift = [ ]
H_drives = [
    ad0 * a0,
    ad1 * a1,
    a0+ad0,
    im * (a0-ad0),
    a1+ad1,
    im * (a1-ad1),
    ad0 * a1 + a0 * ad1
]
system = QuantumSystem(H_drives);

In [ ]:
PICO_max_iter = 250

# Shape the cost function with weights on states and controls
Q = 100.
R = 1e-4

# Add control bounds
a_bound = 2 * π * 100 * MHz
dda_bound = .02
initial = nothing
starter=nothing
infidelities = []
trajectories = []
for i in 0:20
    target = excitation(i*pi/10)
    println("Angle "*string(i*pi/10))
    println("-----------------------")
    p = UnitarySmoothPulseProblem(
        system,
        target,
        n_steps,
        Δt;
        a_bound=a_bound,
        dda_bound=dda_bound,
        Q=Q,
        R=R,
        verbose=true,
        hessian_approximation=true,
        pade_order=10,
        free_time=true,
        timesteps_all_equal=false,
        subspace=[1,2,3,4],
        max_iter=PICO_max_iter,
    )
     if(i==0)
        starter = copy(p.trajectory)
    end
    solve!(p)
    result = copy(p.trajectory)
    initial = result
   
    push!(trajectories,result)
    states = map(iso_vec_to_operator, eachslice(result[:Ũ⃗], dims=2))
    println("Infidelity " *string(unitary_infidelity(result[:Ũ⃗][:, end], operator_to_iso_vec(target))))
    rollout_states = unitary_rollout(result, system; integrator=exp)
    ΔUs = map(norm, eachslice(rollout_states .- result[:Ũ⃗], dims=2))
    println("Rollout Error "*string(maximum(ΔUs)))
    println("Infidelity "*string(unitary_infidelity(rollout_states[:, end], operator_to_iso_vec(target))))
    push!(infidelities,unitary_infidelity(rollout_states[:, end], operator_to_iso_vec(target)))

end

In [ ]:
infidelities

In [ ]:
PICO_max_iter = 250

# Shape the cost function with weights on states and controls
Q = 100.
R = 1e-4

# Add control bounds
a_bound = 2 * π * 100 * MHz
dda_bound = .02
initial = starter
initialized_infidelities = []
initialized_trajectories = []
for i in 0:20
    target=excitation(i*pi/10)
    println("Angle "*string(i*pi/10))
    println("-----------------------")
    p = UnitarySmoothPulseProblem(
        system,
        target,
        n_steps,
        Δt;
        a_bound=a_bound,
        dda_bound=dda_bound,
        Q=Q,
        R=R,
        verbose=true,
        hessian_approximation=true,
        pade_order=10,
        free_time=true,
        timesteps_all_equal=false,
        subspace=[1,2,3,4],
        max_iter=PICO_max_iter,       
    )
    solve!(p, init_traj=initial)
    result = copy(p.trajectory)
    initial = copy(result)
    push!(initialized_trajectories,initial)
    states = map(iso_vec_to_operator, eachslice(initial[:Ũ⃗], dims=2))
    println("Infidelity " *string(unitary_infidelity(initial[:Ũ⃗][:, end], operator_to_iso_vec(target))))
    rollout_states = unitary_rollout(initial, system; integrator=exp)
    ΔUs = map(norm, eachslice(rollout_states .- initial[:Ũ⃗], dims=2))
    println("Rollout Error "*string(maximum(ΔUs)))
    println("Infidelity "*string(unitary_infidelity(rollout_states[:, end], operator_to_iso_vec(target))))
    push!(initialized_infidelities,unitary_infidelity(rollout_states[:, end], operator_to_iso_vec(target)))
end

In [ ]:
initialized_infidelities

In [ ]:
using Interpolations

In [ ]:
initialized_DATA=[]
for theta in range(0,2*pi,2501)
    trajectory = copy(initialized_trajectories[1])
    interp_linear = Interpolations.linear_interpolation( 0:pi/10:2*pi, [i.datavec for i in initialized_trajectories]);
    interp_linear1 = Interpolations.linear_interpolation( 0:pi/10:2*pi, [i.data for i in initialized_trajectories]);
    trajectory.data=interp_linear1(theta)
    trajectory.datavec=interp_linear(theta)
    
    target=excitation(theta)
    states = map(iso_vec_to_operator, eachslice(trajectory[:Ũ⃗], dims=2))
    rollout_states = unitary_rollout(trajectory, system; integrator=exp)
    push!(initialized_DATA,(unitary_infidelity(rollout_states[:, end], operator_to_iso_vec(excitation(theta)))))
end


In [ ]:
DATA=[]
for theta in range(0,2*pi,2501)
    trajectory = copy(trajectories[1])
    interp_linear = Interpolations.linear_interpolation( 0:pi/10:2*pi, [i.datavec for i in trajectories]);
    interp_linear1 = Interpolations.linear_interpolation( 0:pi/10:2*pi, [i.data for i in trajectories]);
    trajectory.data=interp_linear1(theta)
    trajectory.datavec=interp_linear(theta)
    
    target=excitation(theta)
    states = map(iso_vec_to_operator, eachslice(trajectory[:Ũ⃗], dims=2))
    rollout_states = unitary_rollout(trajectory, system; integrator=exp)
    push!(DATA,(unitary_infidelity(rollout_states[:, end], operator_to_iso_vec(excitation(theta)))))
end


In [ ]:
f = Figure()
ax = Axis(f[1, 1],
    title = "Infidelity Log Plot",
    xlabel = "θ",
    ylabel = "Infidelity"
)
lines!(ax, range(0,2*pi,2501), log10.(convert(Array{Float64,1},DATA)), label  = "Non-Seeded",color = :red)
lines!(ax, range(0,2*pi,2501), log10.(convert(Array{Float64,1},initialized_DATA)), label  = "Seeded",color = :blue)
scatter!(ax, range(0,2*pi,21), log10.(convert(Array{Float64,1},initialized_infidelities)), label  = "Computed Seeded",color = :purple)
scatter!(ax, range(0,2*pi,21), log10.(convert(Array{Float64,1},infidelities)), label  = "Computed Non-Seeded",color = :orange)

f[1, 2] = Legend(f, ax, "Infidelity Data", framevisible = false)
f